In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load Oxford Pets dataset from TFDS
(train_ds, test_ds), info = tfds.load('oxford_iiit_pet:3.*.*', split=['train', 'test'], with_info=True)

# Define preprocessing functions
def preprocess_image(data):
    image = tf.image.resize(data['image'], (32, 32))  # Resize image
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    label = tf.image.resize(data['segmentation_mask'], (32, 32))  # Resize label
    label = tf.cast(label, tf.int32)  # Ensure labels are integers
    
    # Convert labels to one-hot categorical
    num_classes = info.features['label'].num_classes
    label = tf.one_hot(label, depth=num_classes)
    
    # Reshape labels
    label = tf.reshape(label, (32, 32, num_classes))  # Reshape to (32, 32, num_classes)
    
    return image, label

# Apply preprocessing to datasets
train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [2]:
for image, label in train_ds.take(1):
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)

Image shape: (32, 32, 3)
Label shape: (32, 32, 37)


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 25s 99ms/step - loss: 1.2329 - accuracy: 0.5305
Epoch 2/100
115/115 [==============================] - 11s 96ms/step - loss: 0.9223 - accuracy: 0.5886
Epoch 3/100
115/115 [==============================] - 11s 100ms/step - loss: 0.8912 - accuracy: 0.5999
Epoch 4/100
115/115 [==============================] - 11s 94ms/step - loss: 0.8794 - accuracy: 0.6110
Epoch 5/100
115/115 [==============================] - 11s 95ms/step - loss: 0.8744 - accuracy: 0.6155
Epoch 6/100
115/115 [==============================] - 11s 98ms/step - loss: 0.8622 - accuracy: 0.6202
Epoch 7/100
115/115 [==============================] - 11s 97ms/step - loss: 0.8658 - accuracy: 0.6204
Epoch 8/100
115/115 [==============================] - 11s 92ms/step - loss: 0.8543 - accuracy: 0.6253
Epoch 9/100
115/115 [==============================] - 12s 101ms/step - loss: 0.8546 - accuracy: 0.6265
Epoch 10/100
115/115 [==============================] - 11s 93ms/step -

In [5]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 5s 38ms/step - loss: 0.8985 - accuracy: 0.5988
Test Loss: 0.898495078086853
Test Accuracy: 0.5987706780433655


Mean IoU: 1.0
Mean Dice Coefficient: 1.7932538751048765
Mean Pixel Accuracy: 0.5804755106125647


In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import keras

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\oxf_32_model.h5")
    
    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.layers[-4].output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 17s 94ms/step - loss: 1.0449 - accuracy: 0.5741
Epoch 2/100
115/115 [==============================] - 11s 93ms/step - loss: 0.9010 - accuracy: 0.5994
Epoch 3/100
115/115 [==============================] - 12s 101ms/step - loss: 0.9052 - accuracy: 0.6057
Epoch 4/100
115/115 [==============================] - 11s 92ms/step - loss: 0.9077 - accuracy: 0.5928
Epoch 5/100
115/115 [==============================] - 11s 95ms/step - loss: 0.9280 - accuracy: 0.5998
Epoch 6/100
115/115 [==============================] - 9s 76ms/step - loss: 0.9049 - accuracy: 0.6066
Epoch 7/100
115/115 [==============================] - 11s 92ms/step - loss: 0.8972 - accuracy: 0.6072
Epoch 8/100
115/115 [==============================] - 11s 94ms/step - loss: 0.8860 - accuracy: 0.6079
Epoch 9/100
115/115 [==============================] - 11s 94ms/step - loss: 0.9062 - accuracy: 0.6037
Epoch 10/100
115/115 [==============================] - 12s 102ms/step - 

In [9]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 6s 36ms/step - loss: 1.4048 - accuracy: 0.5843
Test Loss: 1.4048324823379517
Test Accuracy: 0.5842569470405579


In [10]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import keras

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\rnd_32_model.h5")
    
    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.layers[-4].output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10, monitor='loss', verbose=1)
model.fit(train_ds.batch(32), epochs=100,callbacks=[early_stopping_monitor])

Epoch 1/100
115/115 [==============================] - 16s 95ms/step - loss: 1.0902 - accuracy: 0.5686
Epoch 2/100
115/115 [==============================] - 11s 95ms/step - loss: 0.8914 - accuracy: 0.6006
Epoch 3/100
115/115 [==============================] - 10s 86ms/step - loss: 0.9031 - accuracy: 0.6096
Epoch 4/100
115/115 [==============================] - 11s 92ms/step - loss: 0.9010 - accuracy: 0.6006
Epoch 5/100
115/115 [==============================] - 10s 88ms/step - loss: 0.9058 - accuracy: 0.5953
Epoch 6/100
 82/115 [====================>.........] - ETA: 3s - loss: 0.9401 - accuracy: 0.5894

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)